In [1]:
%matplotlib inline
from tensorscaling import scale, capacity, unit_tensor, random_tensor, marginal, random_unitary, random_orthogonal
import spectral_gap
import numpy as np
import scipy as scipy
import matplotlib.pyplot as plt
from numpy import matrix
from Irene import sdp

# Tensor scaling

Scale 3x3x3 unit tensor to certain non-uniform marginals:

In [ ]:
shape = [3, 3, 3]
targets = [(.5, .25, .25), (.4, .3, .3), (.7, .2, .1)]

res = scale(unit_tensor(3, 3), targets, eps=1e-4)
res

We can also access the scaling matrices and the final scaled state:

In [ ]:
print(res.gs[0], "\n")
print(res.gs[1], "\n")
print(res.gs[2])

Let's now check that the W tensor *cannot* be scaled to uniform marginals:

In [ ]:
shape = [2, 2, 2, 2]
W = np.zeros(shape)
W[1, 0, 0, 0] = W[0, 1, 0, 0] = W[0, 0, 1, 0] = W[0, 0, 0, 1] = .5
targets = [(.5, .5)] * 4

scale(W, targets, eps=1e-4, max_iterations=1000)

To see more clearly what is going on, we can set the `verbose` flag:

In [ ]:
res = scale(W, targets, eps=1e-4, max_iterations=10, verbose=True)

We see that at each point in the algorithm, one of the marginals has Frobenius distance $\approx 0.59$ to being uniform. Indeed, we know that the entanglement polytope of the W tensor does not include the point corresponding to uniform marginals -- see [here](https://www.entanglement-polytopes.org/four_qubits) for an interactive visualization!

# Tuples of matrices and tensors

We can just as well only prescribe the desired spectra for subsystems.
Note that prescribing two out of three marginals amounts to *operator scaling*.

In [ ]:
shape = [3, 3, 3]
targets = [(.4, .3, .3), (.7, .2, .1)]

res = scale(unit_tensor(3, 3), targets, eps=1e-6)
res

Indeed, the last two marginals are as prescribed, while the first marginal is arbitrary.

In [ ]:
print(marginal(res.psi, 0).round(5), "\n")
print(marginal(res.psi, 1).round(5), "\n")
print(marginal(res.psi, 2).round(5))

**Code to make:**

1. Allow different Gaussians. 
2. Convert a psd matrix into a channel (or tensor) in order to scale it
2. Get a tensor scaling algorithm that actually computes the capacity: idea would be to convert to the format where you keep track the scaling the entire time instead of multiplying many times. Then take product of determinants at the end. **done**
3. Exponentiate and graph. **done**
4. Compare to optimal transport




**Convert PSD into channel:** Or exponentiate a tensor. Need to create *some* Kraus for this to work. Can use, e.g., the spectral decomposition of the state. That's probably the right thing - take random unitaries (how to get these? by scaling, I guess). Issue with random unitaries right now: they are shaped incorrectly. How to reshape? 

In [ ]:
#test some reshaping.
b = np.zeros(4)
print(b)
b.reshape([2,2])
u = random_unitary(4)
u.reshape([4,2,2])
# want to get the rows of the unitary to be the Kraus operators. So it's really more like 

maybe the below is total junk...can we always assume kraus operators are orthogonal?

In [30]:
targets = [(.5, .5), (.5,.5)]
#uni = random_unitary(4)
#print(uni)
rando=uni.reshape([4,2,2])
caps = []
#spec = np.random.randn(4)
print("spectrum:", spec)
spec = np.exp(spec)
for j in range(0,20):
    for i in range(0,3):
        rando[i]*=spec[i]
    caps.append(capacity(rando, targets, eps=1e-4, max_iterations=400,randomize=False, verbose=False))
print("eigenvectors:", rando)
print("capacities:")
caps[-1]/len(caps)



spectrum: [-0.08133892  0.71725758 -0.21677825  0.136336  ]
eigenvectors: [[[-8.37549229e-02  1.00185299e-01]
  [ 3.92188996e-02  1.41585419e-01]]

 [[-1.03409571e+06 -1.00123689e+06]
  [ 8.88892113e+05 -1.49469909e+05]]

 [[-5.72547652e-03 -4.03159546e-03]
  [-1.07891740e-02  2.45441435e-03]]

 [[ 5.06465231e-01 -5.45766804e-01]
  [ 8.57743773e-02  6.62022901e-01]]]
capacities:


0.7091940475016523

In [ ]:
np.exp(2*np.array([1,2]))

In [ ]:
4**(1/2)

**Computing the SDP**

In [18]:
b = [1, 1, 2]
#b = [1, -1]


C = [matrix([[1, 0], [0,1]])]
#print(C)
A1 = [matrix([[1, 0], [0, 0]])]
A2 = [matrix([[0, 0], [0, 1]])]
A3 = [matrix([[0, 1], [1, 0]])]

#A3 = [matrix([[2, 0], [0, 3]])]
SDP = sdp('cvxopt')
SDP.SetObjective(b)
SDP.AddConstantBlock(C)
SDP.AddConstraintBlock(A1)
SDP.AddConstraintBlock(A2)
SDP.AddConstraintBlock(A3)
SDP.solve()
print(SDP.Info)
print(SDP)

Optimal solution found.
{'Status': 'Optimal', 'DObj': 2.0, 'PObj': 2.0, 'Wall': 0.0005688667297363281, 'CPU': 0.0005670000000002062, 'y': array([ 1.,  1., -0.]), 'Z': [array([[-0., -0.],
       [-0., -0.]])], 'X': [array([[1., 1.],
       [0., 1.]])], 'solver': 'CVXOPT'}
Semidefinite program with
             # variables:1
           # constraints:3
             with solver:CVXOPT


In [19]:
np.eye(5,1,-1)

array([[0.],
       [1.],
       [0.],
       [0.],
       [0.]])

In [24]:
np.kron(np.kron(np.eye(1,2,1), np.eye(2,1,0)), np.eye(2))

array([[0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [7]:
np.kron(np.eye(3,3), np.eye(1,3,2))

array([[0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.]])

We are going to solve the SDP $\min C \cdot \rho$ subject to $  E_{ij} \otimes I_n \cdot \rho = P_{ij}$ and $ I_n \otimes E_{ij} \cdot \rho = Q_{ij}$ and, of course $\rho \succeq 0$. **For some reason, the below program is coming up infeasible. I think CVXOPT must demand some really strange properties or something, because this thing is definitely feasible. Do we want both primal and dual feasible?**

In [29]:
n = 2
spec = np.random.randn(n**2)
#print(spec)
uni = random_orthogonal(n**2)
#print(uni)
C = uni.dot(np.diag(spec).dot(np.conj(uni).T))
#C= np.eye(n**2,n**2)*3.5
print(C, np.trace(C))
p = np.eye(n,n)/2
#p = np.reshape(p,n**2)
q = np.eye(n,n)/2
#q = np.reshape(q,n**2)
b = []
constraints = []
for i in range(n):
    for j in range(i+1):
        a = np.kron(np.kron(np.eye(1,n,i), np.eye(n,1,-j)), np.eye(n,n))        #c.reshape([n**2,n**2])
        a = (a + a.T)/2
        #print(a)
        constraints.append(matrix(a))
        b.append(p[i,j])
        
        
for i in range(n):
    for j in range(i+1):
        a = np.kron(np.eye(n,n), np.kron(np.eye(1,n,i), np.eye(n,1,-j)))        #c.reshape([n**2,n**2])
        a = (a + a.T)/2
        #print(a)
        constraints.append(matrix(a))
        b.append(q[i,j])


#print(constraints)
#print(len(constraints))

#b = np.append(p,q)

for i in range(len(constraints)):
   print(constraints[i],b[i])
    
#print(b)
SDP = sdp('cvxopt')
SDP.AddConstantBlock([matrix(C)])
SDP.SetObjective(b)
for a in constraints:
    SDP.AddConstraintBlock([a])
#SDP.AddConstantBlock(matrix(C))
#SDP.AddConstraintBlock(a)
#SDP.AddConstraintBlock(constraints)
    


SDP.Option("show_progress",True)
SDP.Option("maxiters",500)
SDP.Option('dualstart',{'zs': matrix(np.eye(4,4))/4})



SDP.solve()
print(SDP.Info)
print(SDP)
        
#print(np.linalg.eigvals(rho))

[[ 0.23367047 -0.26790896 -0.07366509 -0.12066357]
 [-0.26790896  0.16080715  0.19977221  0.23818833]
 [-0.07366509  0.19977221 -0.08993824  0.17077403]
 [-0.12066357  0.23818833  0.17077403  0.25093702]] 0.5554764069488629
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] 0.5
[[0.  0.  0.5 0. ]
 [0.  0.  0.  0.5]
 [0.5 0.  0.  0. ]
 [0.  0.5 0.  0. ]] 0.0
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]] 0.5
[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]] 0.5
[[0.  0.5 0.  0. ]
 [0.5 0.  0.  0. ]
 [0.  0.  0.  0.5]
 [0.  0.  0.5 0. ]] 0.0
[[0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]] 0.5
     pcost       dcost       gap    pres   dres   k/t
 0: -3.1250e-01  5.4158e-01  8e+00  3e+00  5e+00  1e+00
 1:  2.3500e-01  4.5578e-01  8e-01  7e-01  1e+00  4e-01
 2:  7.0068e-01  6.1757e-01  2e-01  1e+00  4e-01  7e-02
 3:  7.5022e-01  6.7994e-01  8e-02  9e-01  2e-01  3e-04
 4:  6.9878e-01  6.9041e-01  2e-02  5e-01  7e-02  1e-03
 5:  6.7942e-01  6.8

332:  6.7975e-01  1.3133e+01  1e-19  3e+00  2e+01  2e-19
333:  6.7975e-01  1.3133e+01  1e-19  3e+00  2e+01  2e-19
334:  6.7975e-01  1.3133e+01  1e-19  3e+00  2e+01  2e-19
335:  6.7975e-01  1.3133e+01  1e-19  3e+00  2e+01  2e-19
336:  6.7975e-01  1.3133e+01  1e-19  3e+00  2e+01  2e-19
337:  6.7975e-01  1.3133e+01  1e-19  3e+00  2e+01  2e-19
338:  6.7975e-01  1.3133e+01  1e-19  3e+00  2e+01  2e-19
339:  6.7975e-01  1.3133e+01  1e-19  3e+00  2e+01  2e-19
340:  6.7975e-01  1.3133e+01  1e-19  3e+00  2e+01  2e-19
341:  6.7975e-01  1.3133e+01  1e-19  3e+00  2e+01  2e-19
342:  6.7975e-01  1.3133e+01  1e-19  3e+00  2e+01  2e-19
343:  6.7975e-01  1.3133e+01  1e-19  3e+00  2e+01  2e-19
344:  6.7975e-01  1.3133e+01  1e-19  3e+00  2e+01  2e-19
345:  6.7975e-01  1.2794e+01  1e-19  3e+00  2e+01  2e-19
346:  6.7975e-01  3.5781e+00  7e-20  3e+00  4e+00  2e-19
347:  6.7975e-01  6.8280e+00  4e-20  3e+00  9e+00  2e-19
348:  6.7975e-01  6.7600e+00  4e-20  3e+00  9e+00  2e-19
349:  6.7975e-01  6.7600e+00  4

Interestingly, the below fails even though it is clearly feasible. I do not understand. Something about linearly independent constraints?

In [48]:
C = matrix([[1]])
#B = [matrix([[1.]]), matrix([[1.]])]
b = [1,1]

Allan = sdp('cvxopt')
Allan.AddConstantBlock([C])
Allan.SetObjective(b)

Allan.AddConstraintBlock([matrix([[1]])])
Allan.AddConstraintBlock([matrix([[1]])])



Allan.solve()
print(Allan.Info)
print(Allan)
Allan.write_sdpa_dat_sparse("please")

{'Status': 'Infeasible', 'solver': 'CVXOPT'}
Semidefinite program with
             # variables:1
           # constraints:2
             with solver:CVXOPT


In [18]:
b = [1, -1, 1]
C = [matrix([[-33, 9], [9, -26]]),
     matrix([[-14, -9, -40], [-9, -91, -10], [-40, -10, -15]])]
A1 = [matrix([[7, 11], [11, -3]]),
      matrix([[21, 11, 0], [11, -10, -8], [0, -8, -5]])]
A2 = [matrix([[-7, 18], [18, -8]]),
      matrix([[0, -10, -16], [-10, 10, 10], [-16, 10, -3]])]
A3 = [matrix([[2, 8], [8, -1]]),
      matrix([[5, -2, 17], [-2, 6, -8], [17, -8, -6]])]
SDP = sdp('cvxopt')
SDP.SetObjective(b)
SDP.AddConstantBlock(C)
SDP.AddConstraintBlock(A1)
SDP.AddConstraintBlock(A2)
SDP.AddConstraintBlock(A3)
SDP.solve()
print(SDP.Info)

     pcost       dcost       gap    pres   dres   k/t
 0: -1.2037e+00 -1.8539e+02  2e+02  1e-16  8e+00  1e+00
 1: -1.2937e+00 -6.8551e+00  5e+00  8e-16  3e-01  3e-02
 2: -2.8964e+00 -3.7331e+00  7e-01  4e-16  4e-02  5e-02
 3: -3.0150e+00 -3.2556e+00  2e-01  8e-16  1e-02  2e-02
 4: -3.1389e+00 -3.1932e+00  5e-02  5e-16  3e-03  5e-03
 5: -3.1533e+00 -3.1547e+00  1e-03  6e-16  7e-05  1e-04
 6: -3.1535e+00 -3.1536e+00  5e-05  2e-16  3e-06  6e-06
 7: -3.1535e+00 -3.1535e+00  1e-06  5e-16  7e-08  2e-07
Optimal solution found.
{'Status': 'Optimal', 'DObj': -3.1535462629831095, 'PObj': -3.153544787970258, 'Wall': 0.013082027435302734, 'CPU': 0.012904999999999944, 'y': array([-0.36766609,  1.89832827, -0.88755043]), 'Z': [array([[15.36293862, 14.02517849],
       [14.02517849, 12.80392252]]), array([[  1.84125998, -12.25250885,  -5.46160959],
       [-12.25250885, 108.33464106,  39.02501485],
       [ -5.46160959,  39.02501485,  16.46864819]])], 'X': [array([[ 0.00396107, -0.00433837],
       [

In [3]:
spectral_gap.main()

scaling tensor of shape (3, 3, 3) and type float64
target spectra:
  0: (0.5, 0.3333333333333333, 0.16666666666666666)
  1: (0.4444444444444444, 0.3333333333333333, 0.2222222222222222)
  2: (0.41666666666666663, 0.3333333333333333, 0.25)
#000: max_dist = 0.48955167 @ sys = 0
#001: max_dist = 0.46154435 @ sys = 1
#002: max_dist = 0.40303336 @ sys = 2
#003: max_dist = 0.14904523 @ sys = 0
#004: max_dist = 0.12500688 @ sys = 1
#005: max_dist = 0.07026794 @ sys = 2
#006: max_dist = 0.04666652 @ sys = 0
#007: max_dist = 0.05548246 @ sys = 1
#008: max_dist = 0.03363745 @ sys = 0
#009: max_dist = 0.02327900 @ sys = 2
#010: max_dist = 0.02301940 @ sys = 1
#011: max_dist = 0.01540601 @ sys = 0
#012: max_dist = 0.01273300 @ sys = 2
#013: max_dist = 0.01280194 @ sys = 1
#014: max_dist = 0.00799569 @ sys = 2
#015: max_dist = 0.00830211 @ sys = 0
#016: max_dist = 0.00506739 @ sys = 1
#017: max_dist = 0.00590542 @ sys = 2
#018: max_dist = 0.00405910 @ sys = 1
#019: max_dist = 0.00368860 @ sys = 0
#0

TypeError: __init__() missing 1 required positional argument: 'cap'